**취미 추천 AI 모델**
> MBTI 성격 유형 테스트를 기반으로 취미를 추천해주는 다중 분류 기반 추천 시스템

>

**AI 모듈 구상도**

<img src = "https://drive.google.com/uc?export=view&id=1vzHVlu8IVOHjWNHG6Z3ODDbah-6YAxyk">

# 데이터셋 구성

* 취미를 선정하고, 각 취미에 대한 검색결과를 크롤링하였다.
* 크롤링한 각 문장들에서 MBTI와 관련된 키워드들을 추출하여 json 형식으로 저장하였다.
* 추출한 키워드들과 MBTI 간의 연관성을 알아보기 위해 유사도를 판단하였다.

In [ ]:
keywords = {
    # 속성 추가 예정
    'E': ["외향 -- --"]
    'I': ["내향 -- --"]
    'N': ["직관 -- --"]
    'S': ["감각 -- --"]
    'F': ["감정 -- --"]
    'T': ["사고 -- --"]
    'J': ["판단 -- --"]
    'P': ["인식 -- --"]
}

In [ ]:
data_crawling = {
    # 속성 추가 예정
    '캠핑': ["-- -- -- --"]
    '야구': ["-- -- -- --"]
    '전시회 관람': ["-- -- -- --"]
    '그림 그리기': ["-- -- -- --"]
    ...
}

**유사도 판단**
> MBTI는 4가지 카테고리(E/I, N/S, F/T, J/P)로 구성되어 있기 때문에, </br>
각 카테고리에서 [E, N, F, J]를 기준으로 키워드 간의 유사도를 비교하여 데이터셋을 재구성하였다.

> 자연어 처리에서 TF-IDF, Cosine 유사도 판단 기법을 사용하였다.
* TF-IDF : 문서 집합에서 한 단어가 얼마나 중요한지를 수치적으로 나타낸 가중치이다.
* Cosine Similarity : 벡터 사이의 각도를 계산하여 두 벡터가 얼마나 유사한지를 측정하는 기법이다.

In [ ]:
# 유사도 판단 함수
def get_score(sentences, hobby, category):
    sentences = data_crawling[hobby]
    s_len = len(sentences)
    compare = keywords[category]
    sentences = compare + sentences
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
    val = cosine_similarity(tfidf_matrix[0:1],tfidf_matrix[1:]).tolist()[0]
    val.sort(reverse=True)
    total = s_len - val.count(0.0)
    try:
        return sum(val)/total
    except:
        return 0

**취미와 MBTI 속성 간의 유사도를 비교하여 재구성한 데이터셋 예시**

In [ ]:
{
    ...
    "캠핑" : [1, 2, 1, 1],
    "야구" : [2, 1, 0, 1],
    "전시회 관람" : [2, 0, 1, 0],
    "그림 그리기" : [1, 0, 0, 1],
    ...
}

# Dataloader
재구성한 JSON 파일에서 데이터를 로드하고 처리한다.
* 각 취미와 MBTI 속성 간의 유사도를 비교한 데이터셋이다.



In [ ]:
import json

class DataLoader:
    def __init__(self, data_path):
        with open(data_path, 'r', encoding='utf8') as f:
            self.data_json = json.load(f)

            self.num2hobby = {}
            self.scores = []
            for idx, key in enumerate(self.data_json):
                self.num2hobby[idx] = key
                self.scores.append(self.data_json[key])

            self.position_score = []
            self.score_bias = []

        def setBias(self, position_score):
            self.position_score = position_score

            for idx in range(len(position_score)):
                if idx == 0:
                    self.score_bias.append(0)
                else:
                    self.score_bias.append(sum(position_score[:idx]))

        def getDatasetWithBias(self):
            scores_with_bias = []
            for score in self.scores:
                score_to_data = [(a+b-1) for a, b in zip(score, self.score_bias)]
                scores_with_bias.append(score_to_data)

            return scores_with_bias

        def getDataWithBias(self, score):
            return [(a+b-1) for a, b in zip(score, self.score_bias)]

        def getNum2Hobby(self):
            return self.num2hobby

        def getLen(self):
            return len(self.scores[0])

        def getCount(self):
            return len(self.scores)


# Train

In [ ]:
import numpy as np
from keras.utils.np_utils import to_categorical
from keras import models
from keras import layers
from dataloader import DataLoader
import argparse
import matplotlib.pyplot as plt

# 문장을 벡터화하는 함수
def vectorize_sequences(sequences, dimension):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        sequence = list(sequence)
        results[i, sequence] = 1.
    return results


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_path', type==str, default='data/dataset.json')
    parser.add_argument('--epoch', type=int, default=100)
    parser.add_argument('--batch_size', type=int, default=5)

    args = parser.parse_args()

    # 데이터 전처리
    dl = DataLoader(args.data_path)
    position_score = [4,4,4,4]
    shape_X = sum(position_score)
    dl.setBias(postion_score)

    X_labels = np.array([i for i in range(dl.getCount())])
    num2hobby = dl.getNum2Hobby()
    scores_with_bias = dl.getDatasetWithBias()

    X_train = vectorize_sequences(scores_with_bias, sum(postion_score))
    one_hot_train_labels = to_categorical(X_labels)

    # 모델 생성
    model = models.Sequential()
    # 신경망 ReLu
    model.add(layers.Dense(40, activation='relu', input_shape=(shape_X,)))
    model.add(layers.Dense(20, activation='relu'))
    # 출력층
    model.add(layers.Dense(dl.getCount(), activation='softmax'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Train
    history = model.fit(X_train,
                        one_hot_train_labels,
                        epochs=args.epoch,
                        batch_size=args.batch_size
                        )

    # Loss 시각화
    loss = history.history['loss']

    epochs = range(1, len(loss) + 1)

    plt.plot(epochs, loss, 'bo', label='train loss')
    plt.title('train and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

    # 모델 저장
    model.save('./hobby_model.h5')

# Prediction

In [ ]:
import numpy as np
import json

def vectorize_sequences(sequences, dimension):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        sequence = list(sequence)
        results[i, sequence] = 1.
    return results

class PredictModule:
    def __init__(self, model):
        self.model = model
        self.score_bias = []
        # 파일명 수정 예정
        with open('취미 json 파일', encoding='utf8') as f:
            hobby_info = json.load(f)
            self.position_score = hobby_info["list"]
            self.num2hobby = hobby_info["hobby_enum"]

        for idx in range(len(self.position_score)):
            if idx == 0:
                self.score_bias.append(0)
            else:
                self.score_bias.append(sum(self.position_score[:idx]))

        def predict(self, user_answer):
            question_type = [0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3]
            predict_score = [0, 0, 0, 0]
            user_mbti = ''
            for q_type, u_answer in zip(question_type, user_answer):
                if u_answer == 2: predict_score[q_type] += 1

            if predict_score[0] >= 2: user_mbti += 'E'
            else: user_mbti += 'I'
            if predict_score[1] >= 2: user_mbti += 'N'
            else: user_mbti += 'S'
            if predict_score[2] >= 2: user_mbti += 'F'
            else: user_mbti += 'T'
            if predict_score[3] >= 2: user_mbti += 'J'
            else: user_mbti += 'P'

            X_predict = [(a+b-1) for a, b in zip(predict_score, self.score_bias)]
            X_predict = vectorize_sequences([X_predict], sum(self.position_score))
            predictions = self.model.predict(X_predict)
            hobby = []
            for pred in predictions:
                index = np.argpartition(pred, -3)[-3:]
                index = index[np.argsort(pred[index])][::-1]
                for i in index:
                    hobby.append(self.num2hobby[str(i)])
            predictResult = {
                "hobbyType":{
                "name":user_mbti
                },
                "hobbies":[
                    {
                        "name":hobby[0]
                    },
                    {
                        "name":hobby[1]
                    },
                    {
                        "name":hobby[2]
                    },
                ]
            }
            return predictResult